# Fit bmimpf6 data

EoS: PR

Parameter T dependence: Constant, 1 parameter - R32,bmimpf6 pair

N (total fitting parameters): 1
-check on initialization

## Import Functions

In [1]:
import idaes

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize

import sys
sys.path.append('../../../')

from pyomo.environ import (Constraint,
                           Var,
                           ConcreteModel,
                           Expression,
                           Param,
                           Objective,
                           SolverFactory,
                           TransformationFactory,
                           value)
from pyomo.opt import TerminationCondition, SolverStatus

from idaes.core import FlowsheetBlock
import idaes.logger as idaeslog
# Import the Generic Parameter Block
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
# Import unit models from the model library
from idaes.generic_models.unit_models import Flash
# Import degrees of freedom tool
from idaes.core.util.model_statistics import degrees_of_freedom

import scipy.stats as stats

## Load Data

In [2]:
data = pd.read_csv('../r32_bmimpf6_subset.csv')

In [3]:
# Model params
params = pd.read_csv('Params/PR_params_1param_Opt1.csv',header=None)

In [4]:
from hfc32_bmimpf6_PR import configuration 

## Apply Parameters + Solve at Each Point

In [5]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [6]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": True,
                           "has_pressure_change": True})

m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(323.15)
m.fs.F101.inlet.pressure.fix(399800)
m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(0.5)

m.fs.properties.PR_kappa_A['R32','bmimpf6'] = float(params[0])

# Initialize the flash unit
m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)

# Fix the state variables on the state block
# m.fs.F101.inlet.pressure.unfix()
m.fs.F101.inlet.mole_frac_comp[0,'bmimpf6'].unfix()

R32_x = np.zeros((len(data)))
P = np.zeros((len(data)))
T = np.zeros((len(data)))

m.fs.liq = Param(mutable=True,default=0.040)
m.fs.liquid = Constraint(expr=m.fs.F101.liq_outlet.mole_frac_comp[0, "bmimpf6"] == m.fs.liq)

for i in range(len(data)):
    print('i:',i)
    print('Try x_IL:',data["x_bmimpf6"].iloc[i])
    m.fs.liq = data["x_bmimpf6"].iloc[i]
    m.fs.F101.inlet.temperature.fix(float(data["temperature"].iloc[i]))
    m.fs.F101.inlet.pressure.fix(float(data["pressure"].iloc[i]))
    m.fs.F101.inlet.mole_frac_comp[0,'R32'].fix(float(data["x_R32"].iloc[i])+0.1)
    m.fs.F101.inlet.mole_frac_comp[0,'bmimpf6'].fix(float(1-(data["x_R32"].iloc[i]+0.1)))
    m.fs.F101.vap_outlet.temperature.fix(float(data["temperature"].iloc[i]))
    
    try:
        m.fs.F101.initialize(outlvl=idaeslog.CRITICAL)
        
    except:
        print('Initialization Error')
        
    try:

        status = solver.solve(m, tee = False)
        
        if (status.solver.status == SolverStatus.ok) and (status.solver.termination_condition == TerminationCondition.optimal):
            print('Feasible/Optimal Solution')
            R32_x[i] = value(m.fs.F101.liq_outlet.mole_frac_comp[0,'R32']) 
            P[i] = value(m.fs.F101.vap_outlet.pressure[0])
            T[i] = data["temperature"].iloc[i]
        else:
            print('Infeasible Solution')
            R32_x[i] = np.nan
            P[i] = np.nan
            T[i] = data["temperature"].iloc[i]
            
    except ValueError:
        R32_x[i] = np.nan
        P[i] = np.nan
        T[i] = data["temperature"].iloc[i]
        print('Solver Error')

i: 0
Try x_IL: 0.974
Feasible/Optimal Solution
i: 1
Try x_IL: 0.894
Feasible/Optimal Solution
i: 2
Try x_IL: 0.729
Feasible/Optimal Solution
i: 3
Try x_IL: 0.572
Feasible/Optimal Solution
i: 4
Try x_IL: 0.442
Feasible/Optimal Solution
i: 5
Try x_IL: 0.35
Feasible/Optimal Solution
i: 6
Try x_IL: 0.185
Feasible/Optimal Solution
i: 7
Try x_IL: 0.982
Feasible/Optimal Solution
i: 8
Try x_IL: 0.924
Feasible/Optimal Solution
i: 9
Try x_IL: 0.81
Feasible/Optimal Solution
i: 10
Try x_IL: 0.703
Feasible/Optimal Solution
i: 11
Try x_IL: 0.61
Feasible/Optimal Solution
i: 12
Try x_IL: 0.525
Feasible/Optimal Solution
i: 13
Try x_IL: 0.448
Feasible/Optimal Solution
i: 14
Try x_IL: 0.372
Feasible/Optimal Solution
i: 15
Try x_IL: 0.994
Feasible/Optimal Solution
i: 16
Try x_IL: 0.953
Feasible/Optimal Solution
i: 17
Try x_IL: 0.887
Feasible/Optimal Solution
i: 18
Try x_IL: 0.826
Feasible/Optimal Solution
i: 19
Try x_IL: 0.769
Feasible/Optimal Solution
i: 20
Try x_IL: 0.715
Feasible/Optimal Solution
i: 21

In [7]:
fitted_data = pd.DataFrame(data=[R32_x,P/1e6,T])
fitted_data = fitted_data.T
fitted_data.columns = ['R32_x','P, MPa','T, K']